# Prompt Templates, intro

CassIO powers a sophisticated set of bindings to seamlessly inject data
from Cassandra tables into your LangChain prompt templates.

## Basic usage

In [1]:
from langchain.prompts.database import CassandraReaderPromptTemplate

This cell simply obtains a `Session` object, i.e. an active connection to your database. Replace with custom code if you are not using Astra DB.

In [2]:
from cqlsession import getCQLSession, getCQLKeyspace
cqlMode = 'astra_db' # 'astra_db'/'local'
session = getCQLSession(mode=cqlMode)
keyspace = getCQLKeyspace(mode=cqlMode)

In [3]:
ctemplate0 = """Please answer a question from a user.
Keep in mind that the user's age is {user_age} and they live in a city with
nickname {city_nickname}.

USER'S QUESTION: {user_question}
YOUR ANSWER:
"""

### Natural binding with the DB

In the (string) template above, some variables are to be filled with a DB lookup.

The following instructions specifies the details of the binding: for instance,
the variable `user_age` is to be found on table `people`, specifically in column `age`:

In [4]:
cassPrompt = CassandraReaderPromptTemplate(
    session=session,
    keyspace=keyspace,
    field_mapper={
        'user_age': ('people', 'age'),
        'city_nickname': ('nickname_by_city', 'nickname'),
    },
    template=ctemplate0,
    input_variables=["user_question"],
    # admit_nulls=False,
)

**Note** that in the command above we specify the _primary key columns_ as `input_variables`, and not the variable names found in the prompt string above.

When formatting the Prompt Template, we will have to specify the primary key values
for the DB lookup -- the rest is done by the prompt template.

In this case there are two lookups from as many tables: the prompt template
takes care of everything, provided you pass all the primary key columns required
across tables.

Note: this operation essentially is a _client-side join_ (a common pattern with Cassandra).

In [5]:
print(cassPrompt.format(city='turin', name='beppe',
                        user_question='Is functional programming fun?'))

Please answer a question from a user.
Keep in mind that the user's age is 2 and they live in a city with
nickname CereaNeh.

USER'S QUESTION: Is functional programming fun?
YOUR ANSWER:



#### Arbitrary row functions

You can specify an arbitrary function to transform the database row into the returned field. The function gets a `{column_name: value}` dictionary expressing the row and returns the value for the prompt template:

In [9]:
def nicknamer(row_dict):
    return f"{row_dict['nickname']} (i.e. {row_dict['city']})"

field_mapper_f = {
    'user_age': ('people', lambda row_dict: row_dict['age'] + 10000),
    'city_nickname': ('nickname_by_city', nicknamer),
}

cassPromptF = CassandraReaderPromptTemplate(
    session=session,
    keyspace=keyspace,
    field_mapper=field_mapper_f,
    template=ctemplate0,
    input_variables=["user_question"],
)

print(cassPromptF.format(city='milan', name='samanta',
                        user_question='Is there a square circle?'))

Please answer a question from a user.
Keep in mind that the user's age is 10014 and they live in a city with
nickname Taaac (i.e. milan).

USER'S QUESTION: Is there a square circle?
YOUR ANSWER:



#### Null and missing values

You can control how the prompt template should behave when a `None` value is encountered or even when a table has no rows altogether for a given primary key.

First, you can pass a boolean parameter `admit_nulls` to the prompt template.

Second, you can use the full four-element tuple format for the entries in the "field mapper". This would be `(table_name, column_name_or_function, admit_nulls, default_value)` (whose `admit_nulls` will override the overall default).

In [17]:
field_mapper_n = {
    'user_age': ('people', 'age'),
    'city_nickname': ('nickname_by_city', 'nickname', True, '(no nickname)'),
}

cassPromptN = CassandraReaderPromptTemplate(
    session=session,
    keyspace=keyspace,
    field_mapper=field_mapper_n,
    template=ctemplate0,
    input_variables=["user_question"],
    admit_nulls=False,
)

# Note: there is no "tokyo" in the nicknames table
print(cassPromptN.format(city='tokyo', name='hideo',
                        user_question='What are we having for lunch?'))

Please answer a question from a user.
Keep in mind that the user's age is 144 and they live in a city with
nickname (no nickname).

USER'S QUESTION: What are we having for lunch?
YOUR ANSWER:



In [20]:
try:
    # Note: there are no rows with city='madrid' in the "people" table
    print(cassPromptN.format(city='madrid', name='alberto',
                             user_question='What are we having for lunch?'))
except Exception as e:
    print(f"Exception => {str(e)}")

Exception => Null data found for user_age
